In [1]:
!pip install feast --quiet
!feast init feature_repo
%cd /content/feature_repo

     |████████████████████████████████| 15.3 MB 25.9 MB/s 
     |████████████████████████████████| 2.3 MB 64.4 MB/s 
     |████████████████████████████████| 11.1 MB 70.8 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 100 kB 10.0 MB/s 
     |████████████████████████████████| 50 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 45.1 MB/s 
     |████████████████████████████████| 45 kB 2.1 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
     |████████████████████████████████| 140 kB 49.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 3.8 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
     |████████████████████████████████| 406 kB 42.9 MB/s 
     |████████████████████████████████| 3.8 MB 45.1 MB/s 
     |████████████████████████████████| 112 kB 50.3 MB/s 
Feast is an open source pr

In [ ]:
import pandas as pd

data_path = './data/driver_stats.parquet'

driver_stats = pd.read_parquet(data_path)
driver_stats.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2022-05-20 07:00:00+00:00,1005,0.928403,0.439460,57,2022-06-04 07:49:04.614
1,2022-05-20 08:00:00+00:00,1005,0.234996,0.436745,630,2022-06-04 07:49:04.614
2,2022-05-20 09:00:00+00:00,1005,0.758373,0.720880,702,2022-06-04 07:49:04.614
3,2022-05-20 10:00:00+00:00,1005,0.722821,0.960819,976,2022-06-04 07:49:04.614
4,2022-05-20 11:00:00+00:00,1005,0.782236,0.075091,66,2022-06-04 07:49:04.614


In [ ]:
!feast apply

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
Traceback (most recent call last):
  File "/usr/local/bin/feast", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in in

In [ ]:
!feast on-demand-feature-views list

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
NAME


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline

from joblib import dump

model = Pipeline(
    steps=[
           ('standard_scaler', StandardScaler()),
           ('classifier', BaggingClassifier(base_estimator=SGDClassifier())),
    ]
)

target = 'label_driver_reported_satisfaction'

X=training_df.drop(columns=[target, 'event_timestamp'])
y=training_df[target]

feature_names = X.columns.to_list()

model.fit(
    X=X,
    y=y
)

dump(model, 'model.jb', compress=3)

['model.jb']

In [ ]:
!feast materialize-incremental {datetime.now().isoformat()}

Materializing 1 feature views to 2022-03-31 16:33:38+00:00 into the sqlite online store.

driver_hourly_stats from 2022-03-30 16:33:40+00:00 to 2022-03-31 16:33:38+00:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 458.04it/s]


In [ ]:
print("--- Data directory ---")
!ls data

--- Data directory ---
driver_stats.parquet  online_store.db  registry.db


In [ ]:
import sqlite3
import pandas as pd

with sqlite3.connect("data/online_store.db") as con:
    print("\n--- Schema of online store ---")
    query = """SELECT COUNT(*) FROM feature_repo_driver_hourly_stats"""
    print(pd.read_sql_query(sql=query, con=con).columns.tolist())


--- Schema of online store ---


,COUNT(*)
0,15


In [ ]:
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    entity_rows=[{'driver_id': id} for id in range(1001, 1006)],
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ]
).to_dict()

pd.DataFrame(feature_vector)

,driver_id,acc_rate,conv_rate,avg_daily_trips
0,1001,0.458063,0.648043,369
1,1002,0.497164,0.019493,265
2,1003,0.579241,0.248530,339
3,1004,0.835490,0.137416,900
4,1005,0.064316,0.009746,264


In [ ]:
X_test = pd.DataFrame(feature_vector)[feature_names]
predicted_satisfaction = model.predict(X=X_test)

best_driver_id = X_test['driver_id'].iloc[np.argmax(predicted_satisfaction)]

print(best_driver_id)

1002


In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

# Entity生成用
def generate_entities():
    return np.random.choice(999999, size=100, replace=False)

# デイリー特徴量生成用
def generate_stats(entities):
    df = pd.DataFrame(columns=["driver_id", "conv_rate", "acc_rate", "avg_daily_trips", "datetime", "created"])
    df['driver_id'] = entities
    df['conv_rate'] = np.random.random(size=100).astype(np.float32)
    df['acc_rate'] = np.random.random(size=100).astype(np.float32)
    df['avg_daily_trips'] = np.random.randint(0, 1000, size=100).astype(np.int32)
    df['datetime'] = pd.to_datetime(
            np.random.randint(
                datetime(2020, 10, 10).timestamp(),
                datetime(2020, 10, 20).timestamp(),
                size=100),
        unit="s"
    )
    df['created'] = pd.to_datetime(datetime.now())
    return df

# リアルタイム特徴量生成用
def generate_trips(entities):
    df = pd.DataFrame(columns=["driver_id", "trips_today", "datetime", "created"])
    df['driver_id'] = entities
    df['trips_today'] = np.random.randint(0, 1000, size=100).astype(np.int32)
    df['datetime'] = pd.to_datetime(
            np.random.randint(
                datetime(2020, 10, 10).timestamp(),
                datetime(2020, 10, 20).timestamp(),
                size=100),
        unit="s"
    )
    df['created'] = pd.to_datetime(datetime.now())
    return df

entities = generate_entities()
stats_df = generate_stats(entities)
trips_df = generate_trips(entities)

In [ ]:
stats_df.head(3)

,driver_id,conv_rate,acc_rate,avg_daily_trips,datetime,created
0,647396,0.050681,0.759853,492,2020-10-11 07:17:31,2022-03-31 17:36:23.098326
1,631350,0.286995,0.184105,953,2020-10-13 14:54:09,2022-03-31 17:36:23.098326
2,81217,0.369156,0.630361,484,2020-10-15 08:12:11,2022-03-31 17:36:23.098326


In [ ]:
trips_df.head(3)

,driver_id,trips_today,datetime,created
0,647396,899,2020-10-11 13:05:30,2022-03-31 17:36:23.101731
1,631350,211,2020-10-17 09:37:36,2022-03-31 17:36:23.101731
2,81217,569,2020-10-19 14:35:20,2022-03-31 17:36:23.101731
